In [2]:
import sys
import os

# Go up one directory (from notebooks/ to OLIST/)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the project root to the Python path
if project_root not in sys.path:
    sys.path.append(project_root)

print(f"Project root added to path: {project_root}")

Project root added to path: d:\My_Projects\OLIST\python


In [3]:
from src.utils import fetch_data_from_bq
import pandas as pd
from src import sql_queries as q
import pandas as pd
import numpy as np
import json


In [4]:
df = fetch_data_from_bq(q.GET_BI_CUSTOMER_COHORTS)
df.head()


BigQuery clients initialized.
Query successful. Scanned 0.004 MB.


,cohort_period,order_period,customers_in_cohort,active_customers,retention_rate,total_spent,period_index,is_weighted_average
0,NaT,NaT,64831,64831,1.0000,10472898.470000000,0,True
1,NaT,NaT,64831,268,0.0041,45126.560000000,1,True
2,NaT,NaT,52523,180,0.0034,27457.370000000,2,True
3,NaT,NaT,46550,129,0.0028,19734.650000000,3,True
4,NaT,NaT,39909,92,0.0023,13920.810000000,4,True


In [ ]:
import pandas as pd
import json
from pathlib import Path

def create_cohort_report(df, path= None):
    """
    Converts a pre-calculated Cohort Analysis DataFrame into a comprehensive
    printed report and a structured JSON file.

    Parameters:
    df : Cohort analysis dataframe with pre-calculated cohort data.
    path : If provided, saves JSON output to this file path. If None, only returns dict.
    """

    # ------------------- 1. Data Separation -------------------
    # Filter the DataFrame based on the helper column to separate the matrix view
    # from the aggregated cross-cohort summary.
    matrix_df = df[df['is_weighted_average'] == False].copy()
    avg_df = df[df['is_weighted_average'] == True].copy()
    
    # --- Define JSON Descriptions ---
    # Define these early so we can print them in the report header
    OVERALL_REPORT_DESCRIPTION = "Customer Cohort Analysis: retention rates (%) and total spending. Each cohort represents customers who made their first purchase in a specific month."
    AVERAGE_BY_PERIOD_INDEX_DESCRIPTION = (
        "Cross-cohort aggregations by period index. The 'period_index' represents months elapsed since that cohort's start. "
        "This metric summarizes the overall health and retention of the entire customer base based on their age (e.g., all 3-month-old customers). "
        "Retention rates are expressed in percentages."
    )
    
    print("="*80)
    print("                      *** Customer Cohort Analysis Report ***")
    print("Note that Retention rates are in percentage and total spending are in USD.")
    print("="*80)

    # ------------------- 2. Overall Summary -------------------
    print("\n### Overall Summary ###")
    # Total distinct cohorts created during the analysis period.
    total_cohorts = matrix_df['cohort_period'].nunique()
    # Total unique customers who made a first purchase (initial cohort size).
    total_customers = int(matrix_df.groupby('cohort_period')['customers_in_cohort'].max().sum())
    # Total revenue generated by all orders within the scope of this cohort analysis.
    total_revenue = float(matrix_df['total_spent'].sum())

    analysis_start = pd.to_datetime(matrix_df['cohort_period'].min()).strftime('%Y-%m')
    analysis_end = pd.to_datetime(matrix_df['order_period'].max()).strftime('%Y-%m')

    print(f"Total Cohorts: {total_cohorts}")
    print(f"Total Customers: {total_customers:,}")
    print(f"Total Revenue: ${total_revenue:,.2f}")
    print(f"Analysis Period: {analysis_start} to {analysis_end}")
    print("-" * 80)

    # Build overall summary JSON block
    overall_summary = {
        "total_cohorts": total_cohorts,
        "total_customers": total_customers,
        "total_revenue": round(total_revenue, 2),
        "analysis_period_start": analysis_start,
        "analysis_period_end": analysis_end
    }

    # ------------------- 3. Cross-Cohort Aggregations (by Period Index) -------------------
    print("\n### Cross-Cohort Aggregation Results (by Period Index) ###")
    # Print the relevant description for context
    print("Cross-cohort aggregations by period index (months elapsed since that cohort's start).")
    print("-" * 80)

    # Select pre-calculated data from the aggregated DataFrame
    avg_display = avg_df[['period_index', 'active_customers', 'retention_rate',
                           'total_spent']].copy()

    # Format retention rate as percentage for display
    avg_display['retention_rate'] = (avg_display['retention_rate'] * 100).round(2)

    avg_display_table = avg_display.copy()
    avg_display_table.columns = ['Period Index', 'Active Customers (Total)', 'Retention % (Cross-Cohort)',
                                 'Total Spent (Aggregated)']

    avg_display_table['Retention % (Cross-Cohort)'] = avg_display_table['Retention % (Cross-Cohort)'].apply(
        lambda x: f"{x}%" if pd.notna(x) else "N/A"
    )
    avg_display_table['Total Spent (Aggregated)'] = avg_display_table['Total Spent (Aggregated)'].apply(
        lambda x: f"${x:,.2f}" if pd.notna(x) else "N/A"
    )

    print(avg_display_table.to_string(index=False))
    print("-" * 80)

    # Build JSON block for cross-cohort aggregations
    aggregated_by_index_month = []
    for _, row in avg_df.iterrows():
        aggregated_by_index_month.append({
            "period_index": int(row['period_index']) if pd.notna(row['period_index']) else None,
            "retention_rate": round(row['retention_rate'] * 100, 2) if pd.notna(row['retention_rate']) else None,
            "total_spent": round(row['total_spent'], 2) if pd.notna(row['total_spent']) else None,
            "total_active_customers": int(row['active_customers']) if pd.notna(row['active_customers']) else None
        })


    # ------------------- 4. Final Output Construction -------------------
    output = {
        "description": OVERALL_REPORT_DESCRIPTION,
        "overall_summary": overall_summary,
        "average_by_period_index": {
            "description": AVERAGE_BY_PERIOD_INDEX_DESCRIPTION,
            "data": aggregated_by_index_month}
    }

    # ------------------- 5. Save and Return -------------------
    if path:
        path_obj = Path(path)
        path_obj.parent.mkdir(parents=True, exist_ok=True)
        with open(path_obj, 'w') as f:
            json.dump(output, f, indent=2, default=str)
        print(f"\n Cohort analysis saved to: {path}")

    return output



In [26]:
create_cohort_report(df)

                      *** Customer Cohort Analysis Report ***
Note that Retention rates are in percentage and total spending are in USD.

### Overall Summary ###
Total Cohorts: 10
Total Customers: 64,831
Total Revenue: $10,604,144.78
Analysis Period: 2017-11 to 2018-09
--------------------------------------------------------------------------------

### Cross-Cohort Aggregation Results (by Period Index) ###
Cross-cohort aggregations by period index (months elapsed since that cohort's start).
--------------------------------------------------------------------------------
 Period Index  Active Customers (Total) Retention % (Cross-Cohort) Total Spent (Aggregated)
            0                     64831                     100.0%           $10,472,898.47
            1                       268                      0.41%               $45,126.56
            2                       180                      0.34%               $27,457.37
            3                       129               

{'description': 'Customer Cohort Analysis: retention rates (%) and total spending. Each cohort represents customers who made their first purchase in a specific month.',
 'overall_summary': {'total_cohorts': 10,
  'total_customers': 64831,
  'total_revenue': 10604144.78,
  'analysis_period_start': '2017-11',
  'analysis_period_end': '2018-09'},
 'average_by_period_index': {'description': "Cross-cohort aggregations by period index. The 'period_index' represents months elapsed since that cohort's start. This metric summarizes the overall health and retention of the entire customer base based on their age (e.g., all 3-month-old customers). Retention rates are expressed in percentages.",
  'data': [{'period_index': 0,
    'retention_rate': 100.0,
    'total_spent': Decimal('10472898.47'),
    'total_active_customers': 64831},
   {'period_index': 1,
    'retention_rate': 0.41,
    'total_spent': Decimal('45126.56'),
    'total_active_customers': 268},
   {'period_index': 2,
    'retention_ra